<a href="https://colab.research.google.com/github/Vineet2107/ML-for-Civil-Engineering-CE784A-/blob/main/Driver_Gaze_Zone_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
#imported necessary packages and libraries
import matplotlib.pyplot as plt
from skimage import io, color
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
import matplotlib.pyplot as plt
% matplotlib inline

In [48]:
#path of train and test data
train_path = '/content/drive/MyDrive/gaze_dataset/train'
test_path = '/content/drive/MyDrive/gaze_dataset/test'

In [49]:
#path of raveling, non revaling and test data
Centerstack_data = '/content/drive/MyDrive/gaze_dataset/train/Centerstack'
Forward_data = '/content/drive/MyDrive/gaze_dataset/train/Forward'
Left_wm_data = '/content/drive/MyDrive/gaze_dataset/train/Left_wing_mirror'
Rearview_data = '/content/drive/MyDrive/gaze_dataset/train/Rearview_mirror'
Right_wm_data = '/content/drive/MyDrive/gaze_dataset/train/Right_wing_mirror'
test_data = '/content/drive/MyDrive/gaze_dataset/test'

In [50]:
list_cs = os.listdir(Centerstack_data)
list_f = os.listdir(Forward_data)
list_lwm = os.listdir(Left_wm_data)
list_rv = os.listdir(Rearview_data)
list_rwm = os.listdir(Right_wm_data)
list_test = os.listdir(test_data)

In [51]:
data_dir = '/content/drive/MyDrive/gaze_dataset/train'


BATCH_SIZE = 32
# IMG_SIZE = (256, 256, 3)

train_dataset = tf.keras.utils.image_dataset_from_directory(data_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            )

Found 2505 files belonging to 5 classes.


In [52]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  batch_size=8)

Found 2505 files belonging to 5 classes.
Using 501 files for validation.


In [60]:
# Apply data augmentation
img_height = 240
img_width = 240
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

ValueError: ignored

In [58]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_dataset.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
image_batch, labels_batch = next(iter(train_ds))

In [66]:
IMG_SIZE = 240

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])

In [67]:
# Apply data augmentation
img_height = 240
img_width = 240
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [68]:
num_classes = 5
model = tf.keras.Sequential([
  
  resize_and_rescale,
  data_augmentation,
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(64, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout((0.2)),
  tf.keras.layers.BatchNormalization(),
  
  tf.keras.layers.Flatten(),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(num_classes,activation = 'softmax')
])

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['aucrc'])

model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=20
)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


79/79 [==============================] - 20s 215ms/step - loss: 87.1362 - accuracy: 0.2028 - val_loss: 18.9844 - val_accuracy: 0.1876
Epoch 2/20
79/79 [==============================] - 16s 203ms/step - loss: 94.3610 - accuracy: 0.1904 - val_loss: 22.0833 - val_accuracy: 0.2275
Epoch 3/20
79/79 [==============================] - 16s 204ms/step - loss: 102.3026 - accuracy: 0.2104 - val_loss: 56.0992 - val_accuracy: 0.1996
Epoch 4/20
79/79 [==============================] - 16s 208ms/step - loss: 87.9971 - accuracy: 0.2240 - val_loss: 58.5679 - val_accuracy: 0.2156
Epoch 5/20
79/79 [==============================] - 16s 203ms/step - loss: 94.8631 - accuracy: 0.2291 - val_loss: 73.2160 - val_accuracy: 0.2136
Epoch 6/20
79/79 [==============================] - 16s 204ms/step - loss: 88.6293 - accuracy: 0.2192 - val_loss: 95.8896 - val_accuracy: 0.1976
Epoch 7/20
79/79 [==============================] - 16s 203ms/step - loss: 94.2839 - accuracy: 0.2188 - val_loss: 90.3540 - val_accuracy: 0.

In [69]:
test_data = "/content/drive/MyDrive/gaze_dataset/Test folder"

In [70]:
train_dataset = tf.keras.utils.image_dataset_from_directory(test_data,
                                                            shuffle=True
                                                            )

Found 509 files belonging to 1 classes.


In [71]:
#predicting on test data X_pred
y_pred = model.predict(val_ds)
y_output = []

In [72]:
y_pred[0]

array([6.1526970e-07, 9.7441092e-02, 5.5233285e-10, 4.1775837e-07,
       9.0255791e-01], dtype=float32)

In [73]:
result = np.where(y_pred[0] == np.amax(y_pred[0]))
print(result[0][0])

4


In [74]:
t = "/content/drive/MyDrive/gaze_dataset/test"

dicti = {}
dicti['filename'] = []
dicti['class'] = []

for f in os.listdir(t):
  ip = os.path.join(t,f)

  image_path = ip
  image = tf.keras.preprocessing.image.load_img(image_path,target_size=(256,256))
  input_arr = tf.keras.preprocessing.image.img_to_array(image)
  input_arr = np.array([input_arr])  # Convert single image to a batch.
  predictions = model.predict(input_arr)

  dicti['filename'].append(f)
  result = np.argmax(predictions,axis = 1)[0]
  if(result == 4):
    dicti['class'].append("Right_wing_mirror")
  elif(result == 3):
    dicti['class'].append("Rearview_mirror")
  elif(result == 2):
    dicti['class'].append("Left_wing_mirror")
  elif(result == 1):
    dicti['class'].append("Forward")
  else:
    dicti['class'].append("Centerstack")

In [75]:
df = pd.DataFrame.from_dict(dicti)

In [76]:
df.head()

,filename,class
0,129.jpg,Right_wing_mirror
1,128.jpg,Rearview_mirror
2,127.jpg,Left_wing_mirror
3,126.jpg,Centerstack
4,121.jpg,Rearview_mirror


In [77]:
df.to_csv("OutputM.csv", index = False)
!cp OutputM.csv "drive/My Drive/"

In [36]:
for i in range(len(y_pred)):
  result = np.where(y_pred[0] == np.amax(y_pred[0]))
  
  if(result == 0):
    y_output.append("Right_wing_mirror")
  if(result == 1):
    y_output.append("Rearview_mirror")
  if(result == 2):
    y_output.append("Left_wing_mirror")
  if(result == 3):
    y_output.append("Forward")
  else:
    y_output.append("Centerstack")

In [40]:
y_output

['Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Centerstack',
 'Center

In [41]:
print(len(list_test))
print(len(y_output))

509
501


In [ ]:
for i in range(len(y_pred)):
  if(y_pred[i][0] > y_pred[i][1]):
    y_output.append("Non_raveling")
  else:
    y_output.append("Raveling")

In [ ]:
# stored non raveling image array in img_nr
img_cs = []
for i in range(len(list_cs)):
  img = io.imread(os.path.join(Centerstack_data,
                               list_cs[i]))
  img_cs.append(np.array(img))
  io.imshow(img)

img_f = []
for i in range(len(list_f)):
  img = io.imread(os.path.join(Forward_data,
                               list_f[i]))
  img_f.append(np.array(img))
  io.imshow(img)

img_lwm = []
for i in range(len(list_lwm)):
  img = io.imread(os.path.join(Left_wm_data,
                               list_lwm[i]))
  img_lwm.append(np.array(img))
  io.imshow(img)

img_rv = []
for i in range(len(list_rv)):
  img = io.imread(os.path.join(Rearview_data,
                               list_rv[i]))
  img_rv.append(np.array(img))
  io.imshow(img)

#stored raveling image array in img_r 
img_rwm = []
for i in range(len(list_rwm)):
  img = io.imread(os.path.join(Right_wm_data,
                               list_rwm[i]))
  img_rwm.append(np.array(img))
  io.imshow(img)

#stored test image array in img_test
img_test = []
for i in range(len(list_test)):
  img = io.imread(os.path.join(test_data,
                               list_test[i]))
  img_test.append(np.array(img))
  io.imshow(img)

In [ ]:
y_cs=np.empty(500)
y_cs.fill(0)

y_f=np.empty(500)
y_f.fill(1)

y_lwm=np.empty(500)
y_lwm.fill(2)

y_rv=np.empty(500)
y_rv.fill(3)

y_rwm=np.empty(500)
y_rwm.fill(4)

y = np.concatenate((y_cs, y_f, y_lwm, y_rv, y_rwm), axis = 0)

In [ ]:
perms = np.random.permutation(2500)

In [ ]:
X = np.concatenate((img_cs, img_f, img_lwm, img_rv, img_rwm), axis = 0)

In [ ]:
X = X[perms]
y = y[perms]